In [219]:
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import requests
import hashlib
import urllib


# TO DO:
- Keep better track of URLs and photo IDs
- Load in historical photo URLs when recrawling

In [294]:
root_img_folder = '/Volumes/Ben\'s Passport/data/Seasons'

global root_img_folder

desired_image_count = 300
search_seasons = ['summer', ] #'autumn', 'winter snow', 'spring'] 


house_terms = ['exterior home', 'exterior house', 'exterior cottage', 'exterior suburban home', 
               'blue house', 'red house', 'yellow house', 'pink house'
               'neighborhood', 'farm', 'forest', 'pennsylvania house', 'vermont house', 'new hampshire house', '']

all_searches = []
for season in search_seasons: 
    for extra_terms in house_terms: 
        search_term = f'{extra_terms} {season} landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting'
        all_searches.append(search_term)

In [289]:
all_searches

['exterior home autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting',
 'exterior house autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting',
 'exterior cottage autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting',
 'exterior suburban home autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting',
 'blue house autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting',
 'red house autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting',
 'yellow house autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting',
 'pink houseneighborhood autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas -lighting',
 'farm autumn landscape  -night -interior -vector -painting -beach -"at autumn" -christmas

In [290]:
def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        
def fetch_image_urls(query: str, target_folder: str, max_links_to_fetch: int, wd: webdriver, image_urls=image_urls, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

        # build the google query

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

#     print('FETCH A:',image_urls)

    image_count = 0
    results_start = 0
    loop_count  = 0
    while (image_count < max_links_to_fetch) and (loop_count < max_links_to_fetch * 3):
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        if results_start == number_results:
            break

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') \
                and 'http' in actual_image.get_attribute('src') \
                and 'https://encrypted-tbn0.gstatic.com' not in actual_image.get_attribute('src'):
                    
                    img_url = actual_image.get_attribute('src')
                    if img_url not in image_urls:
                        
                        # encode the image url to save as the filename
                        md5 = hashlib.md5()
                        md5.update(img_url.encode('utf-8'))
                        hash_name = md5.hexdigest()
                        
                        did_it_work = persist_image(target_folder, img_url, hash_name)
                        
                        if did_it_work == 1:
                            with open(f"{root_img_folder}/image_metadata.txt", 'a') as metadata:
                                metadata.write(str(img_url) + ' ,' + str(hash_name) + '\n')

                            image_urls[img_url] = hash_name
                        
                            image_count += 1
                            break
                    

                    
        if image_count >= max_links_to_fetch:
            print(f"Found: {len(image_urls)} image links, done!")
            break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(10)
            scroll_to_end(wd)
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

            # move the result startpoint further down
            results_start = len(thumbnail_results)
        
        loop_count += 1
#     print('FETCH B:',image_urls)
 

    return image_urls

In [291]:
def persist_image(folder_path:str,url:str, counter):
    try:
        image_content = requests.get(url,timeout=10)
        time.sleep(0.5)
        try:
#         f = open(os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"), 'wb')
#         f.write(image_content)
#         f.close()
            urllib.request.urlretrieve(url, os.path.join(folder_path, 'jpg' + "_" + str(counter) + ".jpg"))
            return 1
        except Exception as e:
            print(f"ERROR - Could not download or save {url} - {e}")
            return 0
    except Exception as e:
        print(f"ERROR - Could not download or save {url} - {e}")
        return 0



In [292]:
def search_and_download(search_term: str, driver_path: str, target_path='./images', number_images=10, image_urls={}):
    target_folder = os.path.join(target_path, '_'.join(search_term.lower().split(' ')))
#     print('SEARCH A:',image_urls)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        image_urls = fetch_image_urls(search_term, target_folder, number_images, wd=wd, image_urls=image_urls, sleep_between_interactions=1)

#     counter = 0
#     for elem in res:
#         persist_image(target_folder, elem, counter)
#         counter += 1
#     print('SEARCH B:',image_urls)

    return image_urls


In [293]:
DRIVER_PATH = './chromedriver'    
image_urls = {}
with open(f"{root_img_folder}/image_metadata.txt", 'r') as metadata:
    lines = metadata.readlines()
    for line in lines:
        url, hash_id = line.split(' ,')
        image_urls[url] = hash_id
        
for search_term in all_searches:
    
    for season_item in search_seasons:
        if season_item in search_term.split('-')[0]:
                season = season_item
    
    save_directory = f'{root_img_folder}/images/{season}'
    image_urls = search_and_download(search_term=search_term, driver_path=DRIVER_PATH, target_path = save_directory, number_images=desired_image_count, image_urls=image_urls)
#     print('MAIN:',image_urls)

Found: 100 search results. Extracting links from 0:100
ERROR - Could not download or save https://p0.piqsels.com/preview/730/315/656/captains-home-military-home-tree-fall.jpg - HTTP Error 403: Forbidden
ERROR - Could not download or save https://images.pexels.com/photos/206673/pexels-photo-206673.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940 - HTTP Error 403: Forbidden
ERROR - Could not download or save https://www.landscape-design-in-a-day.com/wp-content/uploads/2022/06/Front-Yard-entrance-and-sit-spot.jpg - HTTP Error 403: Forbidden
ERROR - Could not download or save https://p2.piqsels.com/preview/519/610/433/germany-landscape-mountains-scenic.jpg - HTTP Error 403: Forbidden
ERROR - Could not download or save https://www.landscape-design-in-a-day.com/wp-content/uploads/2022/06/Backyard-entertaining-with-sun-sail.jpg - HTTP Error 403: Forbidden
ERROR - Could not download or save https://www.monrovia.com/media/amasty/blog/goldenlead1200x675-1024x576_2x.jpg - HTTP Error 403: Forbidde

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: chrome=107.0.5304.87)


In [ ]:
# with open("image_metadata.txt", 'w') as output:
    print()
#     for row in image_urls:  
#         metadata.write(str(row) + ',' + str(image_urls[row]) + '\n')